<a href="https://colab.research.google.com/github/ss1111119/NCCUIndigenousLibrary/blob/main/1_12%E9%9A%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 語言課程字典
language_groups = {
    789: "S01-10 西群卑南語 第十階 學習手冊",
    790: "S02-10 知本卑南語 第十階 學習手冊",
    791: "S03-10 南王卑南語 第十階 學習手冊",
    792: "S04-10 建和卑南語 第十階 學習手冊",
    793: "S05-10 郡群布農語 第十階 學習手冊",
    794: "S06-10 卓群布農語 第十階學 習手冊",
    795: "S07-10 卡群布農語 第十階 學習手冊",
    796: "S08-10 丹群布農語 第十階 學習手冊",
    797: "S09-10 巒群布農語 第十階 學習手冊",
    798: "S10-10 南排灣語 第十階 學習手冊",
    799: "S11-10 東排灣語 第十階 學習手冊",
    800: "S12-10 北排灣語 第十階 學習手冊",
    801: "S13-10 中排灣語 第十階 學習手冊",
    802: "S14-10 霧台魯凱語 第十階 學習手冊",
    803: "S15-10 東魯凱語 第十階 學習手冊",
    804: "S16-10 賽考利克泰雅語 第十階 學習手冊",
    805: "S17-10 澤敖利泰雅語 第十階 學習手冊",
    806: "S18-10 汶水泰雅語 第十階 學習手冊",
    807: "S19-10 萬大泰雅語 第十階 學習手冊",
    808: "S20-10 德固達雅賽德克語 第十階 學習手冊",
    809: "S21-10 德鹿谷賽德克語 第十階 學習手冊",
    810: "S22-10 都達賽德克語(西部) 第十階 學習手冊",
    811: "S23-10 秀姑巒阿美語 第十階 學習手冊",
    812: "S24-10 南勢阿美語 第十階 學習手冊",
    813: "S25-10 海岸阿美語 第十階 學習手冊",
    814: "S26-10 馬蘭阿美語 第十階學 習手冊",
    815: "S27-10 恆春阿美語 第十階 學習手冊",
    816: "S29-10 雅美語 第十階 學習手冊",
    817: "S28-10 賽夏語 第十階 學習手冊",
    818: "S30-10 邵語 第十階 學習手冊",
    819: "S31-10 噶瑪蘭語 第十階 學習手冊",
    820: "S32-10 鄒語 第十階 學習手冊",
    821: "S33-10 卡那卡那富語 第十階 學習手冊",
    822: "S35-10 多納魯凱語 第十階 學習手冊",
    823: "S34-10 拉阿魯哇語 第十階 學習手冊",
    824: "S36-10 萬山魯凱語 第十階 學習手冊",
    825: "S37-10 茂林魯凱語 第十階 學習手冊",
    826: "S41-10 大武魯凱語 第十階 學習手冊",
    827: "S38-10 撒奇萊雅語 第十階 學習手冊",
    828: "S42-10 宜蘭澤敖利泰雅語 第十階 學習手冊",
    829: "S43-10 四季泰雅語 第十階 學習手冊",
    830: "S44-10 太魯閣語 第十階 學習手冊",
    831: "S45-10 都達賽德克語(東部) 第十階 學習手冊"
}


def fetch_and_parse_content():
    for id, name in language_groups.items():
        url = f"https://ebook.alcd.center/learning?view=adm_learning&id={id}"
        response = requests.get(url)
        lesson_data = []

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            for i in range(1, 11):
                lesson_id = f"lesson{i}"
                lesson_div = soup.find('div', id=lesson_id)
                if lesson_div:
                    content = {'Lesson': i, 'Type': [], 'Content': []}

                    # 單詞和發音
                    words, pronunciations = [], []
                    text_words = lesson_div.find_all('div', class_='textWord')
                    for text in text_words:
                        word = text.find('span', class_='cword').text.strip()
                        pronunciation = text.contents[-1].strip(
                        ) if text.contents[-1].name != 'br' else ''
                        words.append(word)
                        pronunciations.append(pronunciation)

                    content['Type'].append('Words')
                    content['Content'].append(words)
                    content['Type'].append('Pronunciations')
                    content['Content'].append(pronunciations)

                    # 翻譯
                    translations = [trans.text.strip() for trans in lesson_div.find_all(
                        'span', class_='cwords')]
                    content['Type'].append('Translations')
                    content['Content'].append(translations)

                    lesson_data.append(content)

            # 建立一個 DataFrame
            rows = []
            for data in lesson_data:
                for content_type, content_list in zip(data['Type'], data['Content']):
                    row = [data['Lesson'], content_type] + content_list
                    rows.append(row)

            # 創建 DataFrame 需要的最大行長度
            max_len = max(len(row) for row in rows)
            df = pd.DataFrame([row + [''] * (max_len - len(row))
                              for row in rows])
            file_name = f"{name}.xlsx".replace("/", " or ")  # 保證文件名符合文件系統規範
            df.to_excel(file_name, index=False, header=False)
        else:
            print(f"無法從 ID {id} 網頁獲取資料")


# 執行函數
fetch_and_parse_content()